In [2]:
import spacy
from spacy import displacy
from spacy.attrs import ORTH, NORM
from bs4 import BeautifulSoup
from pysbd.utils import PySBDFactory
import os
import gzip

import json

In [7]:
DIR_DATA = "../../data/top100/"
DIR_TMP = "../../tmp/"
FILE = "2019-02-19_oldp_cases.json"
CHARACTER_ENCODING = "UTF-8"

In [4]:
nlp = spacy.load("de_core_news_lg")

In [5]:
def addWhitespacesAfterEverySpanTag(soup):
    for index in soup.find_all('span'):
        index.append(" ")
    return soup
    
@nlp.factory("pysbd_sentence_boundaries", default_config={"some_setting": True})
def create_my_component(nlp, name, some_setting):
     return PySBDFactory(nlp, language='de')

In [8]:
with open(os.path.join(DIR_TMP, FILE), mode="r", encoding=CHARACTER_ENCODING) as f:
  for line in f:
    parsed_line = json.loads(line)

    content = parsed_line["content"]

    soup = BeautifulSoup(content, "lxml")

    soup = addWhitespacesAfterEverySpanTag(soup)
    document = nlp(soup.text)

    # add as a spacy pipeline component
    nlp.add_pipe('pysbd_sentence_boundaries', before='parser')

    doc = nlp(document)
    for sent in doc.sents:
        print(sent.text.strip(), "\n")
        print(document)
      # print(createQuintuple(document))

AttributeError: module 'pysbd' has no attribute 'Segmenter'

In [1]:
def handleChildren(quintuple, token):
  for child in token.children:
    if child.dep_ == "sb" or child.dep_ == "oa" or child.dep_ == "da":
        if child.dep_ in quintuple.keys():
            quintuple[child.dep_].append(child.text)
        else:
            quintuple[child.dep_] = [child.text]
        
        quintuple = handleChildren(quintuple, child)

    elif child.dep_ == "cd": 
        if child.dep_ in quintuple['rest'].keys():
            if child.text in quintuple['rest'][child.dep_]:
                quintuple['rest'][child.dep_].append(child.text)
        else:
            quintuple['rest'][child.dep_] = [child.text]
        
        if token.dep_ in quintuple['rest'].keys():
            quintuple = handleChildren(quintuple, child)
        else:
            quintuple[token.dep_] += [x.text for x in list(child.children)]
        
    else:
        if child.dep_ in quintuple['rest'].keys():
            quintuple['rest'][child.dep_].append(child.text)
        else:
            quintuple['rest'][child.dep_] = [child.text]

        quintuple = handleChildren(quintuple, child)
    
  return quintuple

def createQuintuple(sentence):
    for token in sentence:
        if token.dep_ == "ROOT":
            quintuple = {
                'ROOT': [],
                'sb': [],
                'rest': {}
            }

            quintuple[token.dep_].append(token.text)
            quintuple = handleChildren(quintuple, token)

            return quintuple

In [41]:
document = nlp("Der Antrag auf Zulassung der Berufung hat keinen Erfolg.")

In [ ]:
for token in document:
  print(token.text, '\t', token.dep_, '\t', spacy.explain(token.dep_), '\t', token.head.text, '\t', token.head.pos_, '\t',
      [child for child in token.children])